In [1]:
#importar librerias

# Importing libraries
import numpy as np
import pydrive as drive
from google.colab import drive
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
pd.set_option('display.max_rows', 100)

# Modulos para contingencias
from scipy.stats import chi2
from scipy.stats import chi2_contingency

# Modulos de scikit-learn
from sklearn.preprocessing import MinMaxScaler
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
ruta_train = '/content/drive/MyDrive/ProyectoFinalML/train_clear.csv'

train  = pd.read_csv(ruta_train, sep=",", low_memory=True) 
train = train.drop(['Unnamed: 0'], axis=1)
train.head()

,EngineVersion,AppVersion,AvSigVersion,AVProductStatesIdentifier,AVProductsInstalled,CountryIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IeVerIdentifier,Census_MDC2FormFactor,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSBranch,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_ActivationChannel,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsTouchEnabled,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0.556950,0.537115,0.596314,0.758125,0.0,0.126697,0.115254,0.602837,0.895716,0.000000,1.0,0.534936,0.844230,0.317757,1.00000,0.434082,0.026405,0.023622,0.444444,0.522448,5.844540e-08,1.000000,0.013081,0.002350,0.524783,0.105324,0.117259,0.104270,1.000000,1.000000e+00,0.569549,0.527209,0.007603,0.511569,0.512364,1.000000,0.657895,0.732919,0.393503,0.221906,0.574312,0.501200,0.0,0.0,0.0,0.642857,0.0
1,0.401938,0.496127,0.410464,0.758125,0.0,0.416290,0.400000,0.223404,0.895716,0.000000,1.0,0.534936,0.844230,0.317757,0.95943,0.434082,0.265285,0.023622,0.444444,0.536743,5.844540e-08,1.000000,0.004473,0.002350,0.502297,0.076389,0.111238,0.088994,0.916717,2.328306e-10,0.534535,0.527209,0.000046,0.511569,0.512364,0.895791,0.184211,0.186335,0.393503,0.221906,0.574312,0.802386,0.0,0.0,0.0,0.500000,0.0
2,0.556950,0.537115,0.533078,0.758125,0.0,0.384615,0.213559,0.170213,0.895716,0.969697,1.0,0.534936,0.746209,0.317757,1.00000,0.798828,0.919545,0.023622,0.444444,0.440027,1.402780e-08,0.922203,0.004976,0.002350,0.524783,0.120370,0.156319,0.125101,1.000000,1.000000e+00,0.569549,0.527209,0.007603,0.475018,0.475023,1.000000,0.157895,0.180124,1.000000,0.492814,0.128440,0.730592,0.0,0.0,0.0,0.142857,0.0
3,0.556950,0.537115,0.701773,0.758125,0.0,0.393665,0.393220,0.404255,0.895716,0.000000,1.0,0.534936,0.844230,0.317757,1.00000,0.234701,0.798528,0.023622,0.444444,0.507259,2.922331e-08,0.204658,0.009921,0.002350,0.525155,0.103009,0.111238,0.088994,1.000000,1.000000e+00,0.538361,0.527209,0.010505,0.511569,0.512364,1.000000,0.421053,0.391304,1.000000,0.492814,0.323853,0.277967,0.0,0.0,0.0,0.142857,1.0
4,0.556950,0.537115,0.527328,0.758125,0.0,0.076923,0.935593,0.262411,0.895716,0.969697,1.0,0.534936,0.746209,0.317757,0.95943,0.234701,0.960726,0.023622,0.444444,0.557963,5.844540e-08,1.000000,0.004451,0.003653,0.479965,0.076968,0.111238,0.088994,0.916717,0.000000e+00,0.488941,0.527209,0.008801,0.475018,0.475023,0.187473,0.184211,0.186335,1.000000,0.221906,0.323853,0.275109,0.0,0.0,0.0,0.000000,1.0


In [5]:
train.describe()

,EngineVersion,AppVersion,AvSigVersion,AVProductStatesIdentifier,AVProductsInstalled,CountryIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IeVerIdentifier,Census_MDC2FormFactor,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSBranch,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_ActivationChannel,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsTouchEnabled,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
count,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06
mean,5.016512e-01,5.016512e-01,5.016512e-01,6.771519e-01,5.716916e-02,4.851867e-01,5.718901e-01,4.319465e-01,7.795943e-01,6.191962e-01,8.601293e-01,5.016512e-01,7.818303e-01,2.868468e-01,9.534788e-01,3.581122e-01,6.900467e-01,2.348137e-02,3.920571e-01,5.272417e-01,4.290182e-07,9.280948e-01,1.678190e-02,3.615599e-03,5.016512e-01,9.209017e-02,1.261577e-01,1.038058e-01,9.076229e-01,2.531263e-01,5.016512e-01,5.016512e-01,4.574583e-02,5.016512e-01,5.016512e-01,5.818890e-01,3.563367e-01,3.674948e-01,5.648149e-01,2.914324e-01,3.687207e-01,4.579158e-01,5.072511e-01,1.309038e-01,2.930920e-01,4.908132e-01,5.016512e-01
std,6.609164e-02,5.759185e-02,7.809304e-02,2.013509e-01,8.850471e-02,2.854611e-01,3.024099e-01,2.461759e-01,1.817158e-01,4.658642e-01,1.469635e-01,4.874228e-02,5.649784e-02,7.853783e-02,6.935800e-02,2.112278e-01,2.072391e-01,1.577636e-02,1.433856e-01,1.873780e-01,5.870069e-04,2.054752e-01,1.419684e-02,3.019941e-03,2.849333e-02,3.373845e-02,2.946541e-02,2.463212e-02,1.224916e-01,4.348027e-01,4.270749e-02,3.100578e-02,1.366151e-01,2.319963e-02,2.296852e-02,3.779864e-01,2.676185e-01,2.786515e-01,4.163252e-01,1.640810e-01,2.028148e-01,2.940936e-01,4.999475e-01,3.372951e-01,4.551803e-01,3.252805e-01,4.999973e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.920674e-01,4.926996e-01,4.666667e-01,6.880914e-01,0.000000e+00,2.262443e-01,2.983051e-01,2.588652e-01,7.011462e-01,0.000000e+00,8.149000e-01,4.880950e-01,7.462088e-01,2.570093e-01,9.594302e-01,2.347005e-01,5.486473e-01,7.874016e-03,4.444444e-01,4.458343e-01,2.992462e-08,9.222034e-01,5.282185e-03,2.350451e-03,5.022970e-01,7.638889e-02,1.112377e-01,8.899433e-02,9.167168e-01,0.0

In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.drop(['HasDetections'], axis=1), train['HasDetections'], test_size=0.8, train_size=0.2, random_state=42)
del train

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
score=clf.score(X_test, y_test)
print(score)

0.5527584070588482


In [7]:
from lightgbm import LGBMClassifier
clf = LGBMClassifier(objective='binary')
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.6333061976435689

In [56]:
!pip install catboost &> /dev/null
!pip install ipywidgets &> /dev/null
!jupyter nbextension enable --py widgetsnbextension &> /dev/null
!pip install shap &> /dev/null
#!pip install sklearn
!jupyter nbextension enable --py widgetsnbextension
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score

funcion_perdidas = ['Logloss', 'CrossEntropy', 
                    'MultiClassOneVsAll', 'MultiClass',
                    ]
data=[]
for i in funcion_perdidas:
  model = CatBoostClassifier(
      custom_loss=['Accuracy'],
      random_seed=42,
      logging_level='Silent',
      task_type = 'GPU',
      loss_function=i,
      iterations = 5000
  )
  model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    #logging_level='Verbose',  # you can uncomment this for text output
    )
  score = model.score(X_test, y_test)
  data.append([i, score])
categorical_features_indices = np.where(X_train.dtypes != np.float)[0]
model = CatBoostClassifier(
    custom_loss=['Accuracy'],
    random_seed=42,
    logging_level='Silent',
    task_type = 'GPU',
    loss_function='MultiClass',
    iterations = 3000
    #learning_rate=0.04,
    #depth=2
)
model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    logging_level='Verbose',  # you can uncomment this for text output
    plot=True
)

model.score(X_test, y_test)

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.136469
0:	learn: 0.6859070	test: 0.6859340	best: 0.6859340 (0)	total: 14.9ms	remaining: 44.8s
1:	learn: 0.6802800	test: 0.6803057	best: 0.6803057 (1)	total: 28.8ms	remaining: 43.2s
2:	learn: 0.6758488	test: 0.6758629	best: 0.6758629 (2)	total: 42.3ms	remaining: 42.2s
3:	learn: 0.6722477	test: 0.6722543	best: 0.6722543 (3)	total: 56ms	remaining: 41.9s
4:	learn: 0.6693824	test: 0.6693753	best: 0.6693753 (4)	total: 70.4ms	remaining: 42.2s
5:	learn: 0.6670514	test: 0.6670413	best: 0.6670413 (5)	total: 83.7ms	remaining: 41.8s
6:	learn: 0.6650268	test: 0.6650084	best: 0.6650084 (6)	total: 96.6ms	remaining: 41.3s
7:	learn: 0.6634283	test: 0.6634190	best: 0.6634190 (7)	total: 110ms	remaining: 41.3s
8:	learn: 0.6620456	test: 0.6620366	best: 0.6620366 (8)	total: 123ms	remaining: 40.9s
9:	learn: 0.6608317	test: 0.6608182	best: 0.6608182 (9)	total: 135ms	remaining: 40.5s
10:	learn: 0.6598060	test: 0.6598007	best: 0.6598007 (10)	total: 148ms	remaining: 40.2s
11:	learn: 0.6588

0.6438801403309031

In [57]:
model.score(X_test, y_test)

0.6438801403309031

In [58]:
tabla = pd.DataFrame(data=data, columns=['perdida','score'])
tabla.head()

,perdida,score
0,Logloss,0.640528
1,CrossEntropy,0.642141
2,MultiClassOneVsAll,0.641940
3,MultiClass,0.644559


In [59]:
from catboost import Pool, cv



cat_features = [0]

cv_dataset = Pool(data=X_train,
                  label=y_train,
                  #cat_features=cat_features
                  )

params = {"iterations": 100,
          "loss_function": "MultiClass",
          "verbose": True}

scores = cv(cv_dataset,
            params,
            fold_count=2, 
            plot="True")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6915098	test: 0.6915140	best: 0.6915140 (0)	total: 846ms	remaining: 1m 23s
1:	learn: 0.6899413	test: 0.6899470	best: 0.6899470 (1)	total: 1.53s	remaining: 1m 14s
2:	learn: 0.6884592	test: 0.6884757	best: 0.6884757 (2)	total: 2.21s	remaining: 1m 11s
3:	learn: 0.6870526	test: 0.6870717	best: 0.6870717 (3)	total: 2.85s	remaining: 1m 8s
4:	learn: 0.6857387	test: 0.6857592	best: 0.6857592 (4)	total: 3.51s	remaining: 1m 6s
5:	learn: 0.6844677	test: 0.6844917	best: 0.6844917 (5)	total: 4.15s	remaining: 1m 5s
6:	learn: 0.6832752	test: 0.6832987	best: 0.6832987 (6)	total: 4.84s	remaining: 1m 4s
7:	learn: 0.6821345	test: 0.6821585	best: 0.6821585 (7)	total: 5.48s	remaining: 1m 3s
8:	learn: 0.6810447	test: 0.6810735	best: 0.6810735 (8)	total: 6.13s	remaining: 1m 1s
9:	learn: 0.6800256	test: 0.6800537	best: 0.6800537 (9)	total: 6.77s	remaining: 1m
10:	learn: 0.6790409	test: 0.6790702	best: 0.6790702 (10)	total: 7.43s	remaining: 1m
11:	learn: 0.6780893	test: 0.6781143	best: 0.6781143 (1

In [62]:
print(scores)

    iterations  ...  train-MultiClass-std
0            0  ...              0.000030
1            1  ...              0.000056
2            2  ...              0.000117
3            3  ...              0.000149
4            4  ...              0.000175
5            5  ...              0.000235
6            6  ...              0.000259
7            7  ...              0.000261
8            8  ...              0.000276
9            9  ...              0.000261
10          10  ...              0.000282
11          11  ...              0.000287
12          12  ...              0.000291
13          13  ...              0.000295
14          14  ...              0.000294
15          15  ...              0.000287
16          16  ...              0.000316
17          17  ...              0.000334
18          18  ...              0.000331
19          19  ...              0.000365
20          20  ...              0.000402
21          21  ...              0.000402
22          22  ...              0